In [1]:
import os 
from file_handler import open_json, write_file
import matplotlib.pyplot as plt
import matplotlib.markers as markers
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
base_path = "/home/niels/conference"

params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)

original_path = os.path.join(base_path, "original_results")
results_path = os.path.join(base_path, "results")
plots_path = os.path.join(base_path, "plots")

modules = [
    "TOP.mkTbSoc.soc_soc.ccore.dmem.dcache", # Data Cache
    "TOP.mkTbSoc.soc_soc.ccore.riscv.stage2.registerfile", # Register File
    "TOP.mkTbSoc.soc_soc.ccore.riscv.stage5.csr", # CSR
    "TOP.mkTbSoc.soc_soc.ccore.riscv.stage3.multicycle_alu", # ALU
    "TOP.mkTbSoc.soc_soc.ccore.riscv.stage3.multicycle_alu.fpu", #FPU
    "TOP.mkTbSoc.soc_soc.ccore.riscv.stage3.multicycle_alu.mbox", #Mul-Div Unit
    "TOP.mkTbSoc.soc_soc.ccore.riscv.stage0.bpu", # BPU
    "TOP.mkTbSoc.soc_soc.ccore.imem.icache", # Instruction cache
    "TOP.mkTbSoc.soc_soc.ccore.imem.itlb", # Instruction TLB
    "TOP.mkTbSoc.soc_soc.ccore.dmem.dtlb", # Data TLB   
]

file_type = "png"
my_dpi = 10
dpa_b = False

algorithms = ["aes", "des", "des2", "sha3"]

# aes  max timestamps: 152700      skip timestamps: 0 - 100000  => cycle: 0 - 20000
# des  max timestamps: 343810      skip timestamps: 0 - 150000  => cycle: 0 - 30000
# des2 max timestamps: 561390      skip timestamps: 0 - 450000  => cycle: 0 - 90000
# sha3 max timestamps: 252950      skip timestamps: 0 - 0  => cycle: 0 - 0
aes_cycles_skipped = 20000
des_cycles_skipped = 30000
des2_cycles_skipped = 90000
sha3_cycles_skipped = 0

N = 128

marker_colours = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
marker_types = ['o', 'v', 's', 'P', 'X', 'D', 's', '', '*', 'h']


def make_dir(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        print("Directory already exist.")
        
def get_oracles(algorithm):
    if algorithm == algorithms[0]:
        return ["xor", "sboxbyte1", "sboxbyte5", "ro", "fullro"]
    elif algorithm == algorithms[1]:
        return ["column", "or", "row", "sbox"]#, "shift"]
    elif algorithm == algorithms[2]:
        return ["column", "row", "sbox", "and", "or"]
    elif algorithm == algorithms[3]:
        return ["bc", "not", "xor"]
    
def rename_oracle(algorithm, oracle):
    if algorithm == "aes":      
        if oracle == "xor":
            return "sbox_in$_1$"
        if oracle == "sboxbyte1":
            return "sbox_out$_1$"
        if oracle == "sboxbyte5":
            return "sbox_out$_5$"
        if oracle == "fullro":
            return "mc_out"
        if oracle == "ro":
            return "mc_out$_1$"
        
    if algorithm == "des2":
        if oracle == "column":
            return "sbox_column$_1$"
        if oracle == "row":
            return "sbox_row$_1$"
        if oracle == "sbox":
            return "sbox_out$_1$"
        if oracle == "and":
            return "sbox_and$_1$"
        if oracle == "or":
            return "sbox_or$_1$"
    return oracle
def get_cycles_skipped(algorithm):
    if algorithm == algorithms[0]:
        return aes_cycles_skipped
    elif algorithm == algorithms[1]:
        return des_cycles_skipped
    elif algorithm == algorithms[2]:
        return des2_cycles_skipped
    elif algorithm == algorithms[3]:
        return sha3_cycles_skipped

def get_xmin(algorithm):
    if algorithm == algorithms[0]:
        return get_cycles_skipped(algorithm) + 2300
    elif algorithm == algorithms[1]:
        return get_cycles_skipped(algorithm) + 3000
    elif algorithm == algorithms[2]:
        return get_cycles_skipped(algorithm) + 9000
    elif algorithm == algorithms[3]:
        return get_cycles_skipped(algorithm) + 6000
    
    
def get_xmax(algorithm):
    if algorithm == algorithms[0]:
        return 30500
    elif algorithm == algorithms[1]:
        return 70000
    elif algorithm == algorithms[2]:
        return 113000
    elif algorithm == algorithms[3]:
        return 51000
    
def get_unedited_json_results(algorithm, method, module, oracle):
    path = os.path.join(original_path, algorithm, method, "json", module)
    filename = ""
    if method == "svf":
        filename = method + "_" + algorithm + "_" + oracle + "_oracle_" + str(N) + ".json"
    elif method == "tvla":
        filename = method + "_" + algorithm + "_" + str(N) + ".json"
    return open_json(path, filename)

def get_json_results(algorithm, method, module, oracle):
    path = os.path.join(results_path, algorithm, method, "json", module)
    if method == "svf":
        filename = method + "_" + algorithm + "_" + oracle + "_oracle_" + str(N) + ".json"
    elif method == "tvla":
        filename = method + "_" + algorithm + "_" + str(N) + ".json"
    return open_json(path, filename)

def write_json_results(algorithm, method, module, oracle, data):
    path = os.path.join(results_path, algorithm, method, "json", module)
    filename = ""
    if method == "svf":
        filename = method + "_" + algorithm + "_" + oracle + "_oracle_" + str(N) + ".json"
    elif method == "tvla":
        filename = method + "_" + algorithm + "_" + str(N) + ".json"
    write_file(path, filename, data)

In [2]:
# We have skipped uninteresting cycles during svf analysis,
# Increase key values by 'cycles_skipped' so the plots are alligned (svf vs. tvla).
# Opens json file and adds skip_value to every cycle.
# Returns json file.
def change(json_file, skip_value):
    old_values = json_file['values']
    dict_ = {}
    for k in old_values:
        new_k = int(k) + skip_value
        k_value = old_values[k]
        dict_[new_k] = k_value
    json_file['values'] = dict_
    return json_file

def changes(method):
    print("read json from:", original_path)
    print("store json to:", results_path)
    for algorithm in algorithms:
        oracles = get_oracles(algorithm)
        for module in modules:
            if method == "svf":
                for oracle in oracles:
                    path = os.path.join(original_path, algorithm, method, "json", module)
                    new_path = os.path.join(results_path, algorithm, method, "json", module)
                    old_json = get_unedited_json_results(algorithm = algorithm, method = method, module = module, oracle = oracle)
                    new_json = change(old_json, get_cycles_skipped(algorithm))
                    write_json_results(algorithm = algorithm, method = method, module = module, oracle = oracle, data = new_json)
            elif method == "tvla":
                path = os.path.join(original_path, algorithm, method, "json", module)
                new_path = os.path.join(results_path, algorithm, method, "json", module)
                old_json = get_unedited_json_results(algorithm = algorithm, method = method, module = module, oracle = "")
                new_json = old_json
                write_json_results(algorithm = algorithm, method = method, module = module, oracle = "", data = new_json)
    print("finished")

In [3]:
make_dir(results_path)
changes("svf")
changes("tvla")

Directory already exist.
read json from: /home/niels/conference/original_results
store json to: /home/niels/conference/results
finished
read json from: /home/niels/conference/original_results
store json to: /home/niels/conference/results
finished


In [4]:
make_dir(plots_path)

Directory already exist.


In [5]:
def gen_plot_data(_dict):
    keys = []
    values = []
    for t in _dict:
        i = int(t)
        keys.append(i)
        values.append(_dict[t])
    return (keys, values)

In [6]:
def split_leakage(values, threshold_min, threshold_max):
    cycles = []
    out = values.copy()
    for v in values:
        if threshold_max >= values[v] >= threshold_min:
            cycles.append(v) 
    in_ = ([],[])
    greys = []
    for t in cycles:
        i = int(t)
        try:
            in_[1].append(values[t])
            in_[0].append(i)
        except KeyError:
            pass
        try:
            out.pop(t, None)
        except KeyError:
            pass
    return in_, out

In [7]:
def plot_svf(algorithm, module):
    method = "svf"
    m_name = module.split(".")[-1]
    
    oracles = get_oracles(algorithm)
    fig = plt.figure()
    greys = []
    yellows = []
    reds = []
    for i, oracle in enumerate(oracles):
        values = get_json_results(algorithm, method, module, oracle)['values']
        
        grey, out = split_leakage(values, -1.0, 0.299999)
        greys.append(grey)
        yellow, out = split_leakage(values, 0.3, 0.5999)
        yellows.append(yellow)
        red, out = split_leakage(values, 0.6, 1.0)
        reds.append(red)
        
    for idx, x in enumerate(greys):
        plt.scatter(x[0], x[1], marker=marker_types[idx], facecolors='lightgrey', edgecolors="lightgrey")
    
    for idx, x in enumerate(yellows):
        plt.scatter(x[0], x[1], marker=marker_types[idx], facecolors='yellow', edgecolors="black")
        
    for idx, x in enumerate(reds):
        plt.scatter(x[0], x[1], marker=marker_types[idx], facecolors='red', edgecolors="black")
    
    plt.ylabel('SVF score')
    plt.xlabel('Cycle')
    plt.xlim(xmin=get_xmin(algorithm), xmax=get_xmax(algorithm))
    plt.ylim(ymax=1.05)
    plt.axhline(y = 0.6, color = 'darkred', linestyle = ':')
    plt.axhline(y = 0.3, color = 'gold', linestyle = ':')
    lines = []
    for i, oracle in enumerate(oracles):
        oracle = rename_oracle(algorithm, oracle)
        line = mlines.Line2D([], [], marker=marker_types[i], label=oracle, linewidth=0, color='black', fillstyle="none")
        lines.append(line)
    plt.legend(handles=lines)
    #plt.legend()

    file_path = os.path.join(plots_path, method + "_" + algorithm + "_" + m_name + "." + file_type)
    plt.savefig(file_path, format=file_type)
    plt.cla()
    plt.clf()
    plt.close()
    
#plot_svf(algorithms[0], modules[2])

In [8]:
def plot_tvla(algorithm, module):
    fig = plt.figure()
    m_name = module.split(".")[-1]
    
    # Get TVLA values
    tvla_values = get_json_results(algorithm, "tvla", module, "")['values']
    tvla_noleaks, tvla_leaks = split_leakage(tvla_values, -4.49999999, 4.4999999999)
    
    k2, v2 = gen_plot_data(tvla_leaks)
    plt.scatter(k2, v2,  marker=".",  facecolors='royalblue', edgecolors="royalblue")
    plt.scatter(tvla_noleaks[0], tvla_noleaks[1],  marker=".",  facecolors='lightgrey', edgecolors="lightgrey")
        
    plt.ylabel('t-score')
    plt.xlabel('Cycle')
    plt.xlim(xmin=get_xmin(algorithm), xmax=get_xmax(algorithm))
    plt.axhline(y = 4.5, color = 'darkred', linestyle = ':')
    plt.axhline(y = -4.5, color = 'darkred', linestyle = ':')
    
    lines = []
    line1 = mlines.Line2D([], [], marker="o", label="leaky point", linewidth=0, color='royalblue')
    line2 = mlines.Line2D([], [], marker="o", label="non-leaky point", linewidth=0, color='lightgrey')
    lines.append(line1)
    lines.append(line2)
    plt.legend(handles=lines)
    
    file_path = os.path.join(plots_path,  "tvla_" + algorithm + "_" + m_name + "." + file_type)
    plt.savefig(file_path, format=file_type)
    plt.cla()
    plt.clf()
    plt.close()
    
#plot_tvla(algorithms[0], modules[2])

In [9]:
def plot_svf_tvla(algorithm, module):
    fig = plt.figure()
    m_name = module.split(".")[-1]
    # Get TVLA values
    tvla_values = get_json_results(algorithm, "tvla", module, "")['values']
    tvla_noleaks, tvla_leaks = split_leakage(tvla_values, -4.49999999, 4.4999999999)
    k2, v2 = gen_plot_data(tvla_leaks)
    plt.scatter(k2, v2,  marker=".", facecolors='royalblue', edgecolors="royalblue")
    plt.scatter(tvla_noleaks[0], tvla_noleaks[1],  marker=".",  facecolors='lightgrey', edgecolors="lightgrey")
    oracles = get_oracles(algorithm)
    for idx, oracle in enumerate(oracles):
        # Get SVF values of oracle.
        svf_values = get_json_results(algorithm, "svf", module, oracle)['values']
        
        # Find yellow cycles and red cycles of oracle.
        yellow_cycles,_ = split_leakage(svf_values, 0.3, 0.59999999999999)[0]
        red_cycles,_ = split_leakage(svf_values, 0.6, 1.0)[0]

        
        # Color these yellow and red cycles in tvla.
        k = []
        v = []
        for yellow_cycle in yellow_cycles:
            try:
                tvla_value = tvla_values[str(yellow_cycle)]
                k.append(yellow_cycle)
                v.append(tvla_value)
            except KeyError:
                pass 
        plt.scatter(k, v,  marker=marker_types[idx], facecolors='yellow', edgecolors="black")   
        
        red_keys = []
        red_values = []
        for red_cycle in red_cycles:
            try:
                tvla_value = tvla_values[str(red_cycle)]
                red_keys.append(red_cycle)
                red_values.append(tvla_value)
            except KeyError:
                pass 
        plt.scatter(red_keys, red_values,  marker=marker_types[idx], facecolors='red', edgecolors="black") 
        
    lines = []
    for i, oracle in enumerate(oracles):
        oracle = rename_oracle(algorithm, oracle)
        line = mlines.Line2D([], [], marker=marker_types[i], label=oracle, linewidth=0, color='black', fillstyle="none")
        lines.append(line)
    plt.legend(handles=lines, loc=1)
    plt.ylabel('t-score')
    plt.xlabel('Cycle')
    plt.xlim(xmin=get_xmin(algorithm), xmax=get_xmax(algorithm))
    plt.axhline(y = 4.5, color = 'darkred', linestyle = ':')
    plt.axhline(y = -4.5, color = 'darkred', linestyle = ':')
    
    file_path = os.path.join(plots_path,  "svf_tvla_" + algorithm + "_" + m_name + "." + file_type)
    plt.savefig(file_path, format=file_type)
    plt.cla()
    plt.clf()
    plt.close()
#plot_svf_tvla(algorithms[0], modules[2])

In [10]:
for algorithm in algorithms:
    for module in modules:
        plot_svf(algorithm, module)

for algorithm in algorithms:
    for module in modules:
        plot_tvla(algorithm, module)
        
for algorithm in algorithms:
    for module in modules:
        plot_svf_tvla(algorithm, module)

In [16]:
# Data for LaTeX table.
def table_gen():
    algorithm = "des2"
    method = "svf"

    #oracles = ["column", "or", "row"]
    oracles = get_oracles(algorithm)[3:]
    print(oracles)
    for module in modules:
        m_name = module.split(".")[-1].upper()
        s = "\\textbf{" + m_name + "}\t"
        for oracle in oracles:
            s = s
            data = get_json_results(algorithm, method, module, oracle)
            #print(rename_oracle(algorithm, oracle))
            max_value = data['max_svf']
            leaks = data['leak_count']
            s = s + " & " + str(round(max_value, 2)) + " & " + str(leaks)
        s = s + " \\\\"
        print(s)
table_gen()

['and', 'or']
\textbf{DCACHE}	 & 0.11 & 0 & 0.11 & 0 \\
\textbf{REGISTERFILE}	 & 0.36 & 2 & 0.36 & 2 \\
\textbf{CSR}	 & 0.41 & 4 & 0.41 & 4 \\
\textbf{MULTICYCLE_ALU}	 & 0.74 & 6 & 0.74 & 6 \\
\textbf{FPU}	 & 0.74 & 6 & 0.74 & 6 \\
\textbf{MBOX}	 & 0.74 & 10 & 0.74 & 10 \\
\textbf{BPU}	 & 0.01 & 0 & 0.01 & 0 \\
\textbf{ICACHE}	 & 0.01 & 0 & 0.01 & 0 \\
\textbf{ITLB}	 & 0.01 & 0 & 0.01 & 0 \\
\textbf{DTLB}	 & 0.04 & 0 & 0.04 & 0 \\
